In [1]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter
import pandas_datareader.data as web

In [2]:
end = datetime.today()
#start = end - relativedelta(years=2) - relativedelta(days=3)
start = end - relativedelta(years=2)

In [3]:
print(end.strftime('%Y-%m-%d'))
print(start.strftime('%Y-%m-%d'))

2022-06-09
2020-06-09


In [4]:
index_names = ['SP1500', 'SP500', 'SP600', 'SP400']
indexes = ['SPTM','SPY','SLY','MDYG']
sector_names = ['Communication Services ETF', 'Consumer Discretionary ETF', 'Consumer Staples ETF', 'Energy ETF', 'Financials ETF', 'Health Care ETF', 'Industrials ETF', 'Information Technology ETF', 'Materials ETF', 'Real Estate ETF', 'Utilities ETF']
sectors = ['VOX','VCR','VDC','VDE','VFH','VHT','VIS','VGT','VAW','VNQ','VPU']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DTB3",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1250881600Q",\
"LEU0250918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MZMSL",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WTB3MS"    
]

etf_bonds = [*etf_bonds,*etf_muni_bonds,*etf_treasuries]

In [5]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [6]:
sp500 = SP500ticker_list['Symbol']
sp500 = [w.replace('.', '-') for w in sp500]
sp600 = SP600ticker_list['Ticker symbol']
sp600 = [w.replace('.', '-') for w in sp600]
sp400 = SP400ticker_list['Ticker symbol']
sp400 = [w.replace('.', '-') for w in sp400]


In [7]:
len(sp500)+len(sp600)+len(sp400)

1505

In [8]:
print(len(sp500))
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

504
601
400


In [9]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

504
601
400


In [10]:
len(sp1500_index_df)-len(SP1500)

0

In [11]:
def Fred_Data(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    temp = web.DataReader(str(name), 'fred', start_, end_)
    temp.index = pd.to_datetime(temp.index)
    temp.columns = ['Value']
    temp['Symbol'] = name
    #temp = temp.resample(frequency).mean().dropna()
    return(temp)

def dl(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    subset = yf.download(name, start=start_, end=end_, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset['Symbol'] = name

    return (subset)
    
def dl2(assets):

    yahoo_financials = YahooFinancials(assets)

    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [12]:
def getStock(npa):
    #position = npa[0]
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    data_ = yf.download(symbol, start=start_,end=end_)
    #data_['Sector'] = tickerdata.info['sector']
    #data_['Symbol'] = symbol
    #data_['Market Cap'] = data.get_quote_yahoo(names[n])['marketCap']
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [13]:
def getTickerData(npa):
    symbol = npa[0]
    
    stock_data = yf.Ticker(symbol) 
    return([symbol,stock_data])

In [14]:
def getMarketData(npa):
    symbol = npa[0]

    #start_=npa[2]
    #end_=npa[3]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    #data_ = data.get_quote_yahoo(symbol)['marketCap']
    data_ = data.get_quote_yahoo(symbol)
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [15]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    
    if 'sector' in tickerdata.info:
        sector = tickerdata.info['sector']
    else:
        sector = 'error'

    if 'industry' in tickerdata.info:
        industry = tickerdata.info['industry']
    else:
        industry = 'error'

    return([symbol,sector,industry])

In [16]:
def derive_price_supply_trends(npa_):
    name = npa_[0]
    data = npa_[1]
    data_ = data
    data_['Symbol'] = name
    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    
    data_['trailing_1yr_max'] = (pd.DataFrame(data_)['High']).rolling(250).max()
    data_['MIN_Lookback_One_Year'] = data_['trailing_1yr_max'].rolling(250).min()
    data_['MIN_Lookback_6_Months'] = data_['trailing_1yr_max'].rolling(125).min()
    data_['MIN_Lookback_2_Months'] = data_['trailing_1yr_max'].rolling(42).min()
    data_['MIN_Lookback_1_Months'] = data_['trailing_1yr_max'].rolling(21).min()

    data_['trailing_1yr_min'] = (pd.DataFrame(data_)['Low']).rolling(250).min()
    
    data_['supply_trend_1yr'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(250).min()
    
    data_['supply_trend_180d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(125).min()
    
    data_['supply_trend_90d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(63).min()
    
    data_['supply_trend_60d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(42).min()
    
    data_['supply_trend_30d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(21).min()

    data_['trailing_60d_max'] = (pd.DataFrame(data_)['High']).rolling(42).max()
    data_['trailing_60d_min'] = (pd.DataFrame(data_)['Low']).rolling(42).min()
    
    data_['trailing_30d_max'] = (pd.DataFrame(data_)['High']).rolling(21).max()
    data_['trailing_30d_min'] = (pd.DataFrame(data_)['Low']).rolling(21).min()

    data_['trailing_risk_40d_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
    data_['trailing_risk_40d_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
    
    data_['risk_per_share'] = data_['trailing_risk_40d_max']-data_['trailing_risk_40d_min']
    
    data_['30d_vol'] = (pd.DataFrame(data_)['Volume']).rolling(21).sum()
    
    data_['momentum_1yr'] = (data_['Adj Close']-data_['MIN_Lookback_One_Year']) + (data_['Adj Close']-data_['MIN_Lookback_6_Months']) + (data_['Adj Close']-data_['MIN_Lookback_2_Months']) + (data_['Adj Close']-data_['MIN_Lookback_1_Months'])
    
    data_['risk_trend_factor'] = data_['momentum_1yr']/data_['risk_per_share']
    
    return(data_)

In [17]:
def extractTickerValues(npa):
    n_ = npa[0]
    r_ = npa[1]
    #print(r_)
    print(n_)
    
    subset = pd.DataFrame([r_.info])
    
    trackers = ['beta','currentRatio','debtToEquity','dividendRate','dividendYield','earningsGrowth','ebitda','ebitdaMargins','forwardPE','freeCashflow','grossMargins','grossProfits','heldPercentInstitutions','industry','marketCap','operatingCashflow','operatingMargins','payoutRatio','pegRatio','priceToBook','priceToSalesTrailing12Months','profitMargins','quickRatio','returnOnAssets','returnOnEquity','revenueGrowth','revenuePerShare','sector','shortRatio','totalAssets','totalCash','totalCashPerShare','totalDebt','totalRevenue','trailingPE']
    values = pd.DataFrame()
    
    for t in trackers:
        if(np.sum(np.where(subset.columns==t))==0):
            values = pd.concat([values,pd.DataFrame('error',index=[n_],columns=[t])],axis=1)
        else:
            values = pd.concat([values,pd.DataFrame(subset[t].values[0],index=[n_],columns=[t])],axis=1)

    return([n_,values])

In [18]:
def calculateMetrics(data_):
    data_df = data_
    n_ = data_df['Symbol'].values[-1]
    
    latest_p = pd.DataFrame(data_df)['Adj Close'][-1]
    oneYearDate = pd.DataFrame(data_df)['Adj Close'][[-1]].index[0] - relativedelta(years=1)
    
    #separate subset for past year.
    past_year = data_df[data_df.index>=oneYearDate.strftime('%Y-%m-%d')]
    
    #np.max(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['High'])
    max_2y = pd.DataFrame(data_df)['High'].rolling(500).max()[-1]
    min_2y = pd.DataFrame(data_df)['Low'].rolling(500).min()[-1]
    
    #quantiles_close_2y = data_df['Adj Close'].rolling(500).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_2y = data_df['Adj Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_2y = pd.DataFrame(data_df)['Adj Close'].rolling(500).std()[-1]
    mean_2y = pd.DataFrame(data_df)['Adj Close'].rolling(500).mean()[-1]
    
    max_1y = past_year['High'].rolling(250).max()[-1]
    min_1y = past_year['Low'].rolling(250).min()[-1]
    
    #quantiles_close_1y = data_df['Adj Close'].rolling(250).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_1y = past_year['Adj Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_1y = past_year['Adj Close'].rolling(250).std()[-1]
    mean_1y = past_year['Adj Close'].rolling(250).mean()[-1]
    
    risk_trend_factor = data_df['risk_trend_factor'][-1]
    
    supply_trend_1yr = data_df['supply_trend_1yr'][-1]
    supply_trend_180d = data_df['supply_trend_180d'][-1]
    supply_trend_90d = data_df['supply_trend_90d'][-1]
    supply_trend_60d = data_df['supply_trend_60d'][-1]
    supply_trend_30d = data_df['supply_trend_30d'][-1]
    
    MIN_Lookback_One_Year = data_df['MIN_Lookback_One_Year'][-1]
    MIN_Lookback_6_Months = data_df['MIN_Lookback_6_Months'][-1]
    MIN_Lookback_2_Months = data_df['MIN_Lookback_2_Months'][-1]
    MIN_Lookback_1_Months = data_df['MIN_Lookback_1_Months'][-1]
    
    #vol_30d_2yr = data_df['30d_vol'].rolling(500).mean()[-1]
    vol_30d_2yr = np.mean(data_df['30d_vol'])
    volume_factor = data_df['30d_vol'][-1]/vol_30d_2yr
    
    pct_2yr = (latest_p - min_2y)/(max_2y-min_2y)
    pct_1yr = (latest_p - min_1y)/(max_1y-min_1y)
    
    return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend_factor,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr,MIN_Lookback_One_Year, MIN_Lookback_6_Months, MIN_Lookback_2_Months, MIN_Lookback_1_Months,volume_factor])

In [19]:
#test
"""
dates = []
for d in pd.read_csv('sample.csv')['Date']:
    dates.append(datetime.strptime(d, '%d-%b-%y'))

temp = pd.read_csv('sample.csv',thousands=",")
temp.index = dates
temp = temp.iloc[:,1:]
temp.columns = ['Open','High','Low','Adj Close','Volume']
#temp['Close'] = temp['Close'].astype(float)
temp.sort_index(inplace=True)
temp

derive_price_supply_trends(['test',temp]).sort_index(ascending=False)
calculateMetrics(temp)
"""

'\ndates = []\nfor d in pd.read_csv(\'sample.csv\')[\'Date\']:\n    dates.append(datetime.strptime(d, \'%d-%b-%y\'))\n\ntemp = pd.read_csv(\'sample.csv\',thousands=",")\ntemp.index = dates\ntemp = temp.iloc[:,1:]\ntemp.columns = [\'Open\',\'High\',\'Low\',\'Adj Close\',\'Volume\']\n#temp[\'Close\'] = temp[\'Close\'].astype(float)\ntemp.sort_index(inplace=True)\ntemp\n\nderive_price_supply_trends([\'test\',temp]).sort_index(ascending=False)\ncalculateMetrics(temp)\n'

In [20]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [21]:
npa = []
for i in range(0,len([*SP1500,*indexes,*sectors])):
    npa.append([[*SP1500,*indexes,*sectors][i]])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getTickerData, npa)

results_ticker_data = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_data.append("error")
    else:
        results_ticker_data.append(f.result()) 
client.restart()
client.close()


In [22]:
print(results_ticker_data[0][1].info)
print(results_ticker_data[0][1].quarterly_balance_sheet)
print(results_ticker_data[0][1].quarterly_cashflow)
#print(results_ticker_data[0][1].quarterly_dividends)
print(results_ticker_data[0][1].quarterly_earnings)
print(results_ticker_data[0][1].quarterly_financials)


{'zip': '60061', 'sector': 'Technology', 'fullTimeEmployees': 13900, 'longBusinessSummary': 'CDW Corporation provides information technology (IT) solutions in the United States, the United Kingdom, and Canada. It operates through three segments: Corporate, Small Business, and Public. The company offers discrete hardware and software products and services, as well as integrated IT solutions, including on-premise, hybrid, and cloud capabilities across data center and networking, digital workspace, and security. Its hardware products comprise notebooks/mobile devices, network communications, desktop computers, video monitors, enterprise and data storage, and others; and software products consists of application suites, security, virtualization, operating systems, and network management. The company also provides advisory and design, software development, implementation, managed, professional, configuration, and telecom services, as well as warranties; mission critical software, systems, a

In [23]:
results_ticker_data[0][1].info

{'zip': '60061',
 'sector': 'Technology',
 'fullTimeEmployees': 13900,
 'longBusinessSummary': 'CDW Corporation provides information technology (IT) solutions in the United States, the United Kingdom, and Canada. It operates through three segments: Corporate, Small Business, and Public. The company offers discrete hardware and software products and services, as well as integrated IT solutions, including on-premise, hybrid, and cloud capabilities across data center and networking, digital workspace, and security. Its hardware products comprise notebooks/mobile devices, network communications, desktop computers, video monitors, enterprise and data storage, and others; and software products consists of application suites, security, virtualization, operating systems, and network management. The company also provides advisory and design, software development, implementation, managed, professional, configuration, and telecom services, as well as warranties; mission critical software, systems

In [24]:
npa = []
results_ticker_names = [r[0] for r in results_ticker_data]
for n in range(0,len(results_ticker_names),1):
    npa.append([results_ticker_names[n],results_ticker_data[n][1]])

In [25]:

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(extractTickerValues, npa)

results_ticker_values = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_values.append("error")
    else:
        results_ticker_values.append(f.result()) 
client.restart()
client.close()



C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [26]:
ticker_values = pd.DataFrame()

for r in results_ticker_values:
    if(r=='error'):
        pass
    else:                
        #print(r[1])
        subset = pd.DataFrame(r[1])
        subset.index = [r[0]]
        #print(subset)
        ticker_values = pd.concat([ticker_values,subset],axis=0)
        #ticker_values = pd.concat([ticker_values,pd.DataFrame(r[1],index=[r[0]])],axis=0)

In [27]:
#market_data = pd.concat([ticker_values,sectors_df.loc[~sectors_df.index.duplicated(), :].reindex(mktcap_df.loc[~mktcap_df.index.duplicated(), :].index)],axis=1)
market_data = pd.concat([ticker_values])

In [28]:
#market_data.sort_values(by='marketCap',ascending=False)

In [29]:
#market_data

In [30]:
#total_mkt_cap = mktcap_df.replace([np.nan], 0).sum()
#mktcap_df['mkt_cap_pct'] = mktcap_df.replace([np.nan], 0)/total_mkt_cap

In [31]:
#[rp[2] for rp in results_prices][0].info['sector']

In [32]:
"""
arr=np.array(names_prices)
pd.Series(arr).value_counts()
set_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]
if(len(set_)==0):
    duplicates = []
    pass
else:
    duplicates = set_.index[0]
    """

'\narr=np.array(names_prices)\npd.Series(arr).value_counts()\nset_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]\nif(len(set_)==0):\n    duplicates = []\n    pass\nelse:\n    duplicates = set_.index[0]\n    '

In [33]:
#len(results)

In [34]:
names_prices = [rp[0] for rp in results_prices]

#def process_df(data_):
npa = []    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    n_ = names_prices[n]
    npa.append([n_,results_prices[n][1]])
    
prices_df = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result()) 
        prices_df = pd.concat([f.result(),prices_df],axis=0)
client.close()

In [35]:
#np.unique(np.array(prices_df['Symbol'].values))

In [36]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [37]:
#import matplotlib.pyplot as plt
#plt.plot(prices_df[prices_df['Symbol']=='VOX']['Adj Close'])

In [38]:
prices_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol',
       'trailing_1yr_max', 'MIN_Lookback_One_Year', 'MIN_Lookback_6_Months',
       'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months', 'trailing_1yr_min',
       'supply_trend_1yr', 'supply_trend_180d', 'supply_trend_90d',
       'supply_trend_60d', 'supply_trend_30d', 'trailing_60d_max',
       'trailing_60d_min', 'trailing_30d_max', 'trailing_30d_min',
       'trailing_risk_40d_max', 'trailing_risk_40d_min', 'risk_per_share',
       '30d_vol', 'momentum_1yr', 'risk_trend_factor'],
      dtype='object')

In [39]:
#[rp[0] for rp in results_prices]

In [40]:
npa = []
for s in names_prices:
    #print(s)
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    

In [41]:
screenerMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()

missing_screener_metrics = list(set([*SP1500,*indexes,*sectors]).difference([n for n in pd.DataFrame(screenerMetrics).set_index(0).index]))
print(missing_screener_metrics)

for r in range(0,len(missing_screener_metrics)):
    r_ = missing_screener_metrics[r]
    test = np.array([r_,*np.repeat('missing',len(pd.DataFrame(screenerMetrics).set_index(0).iloc[[0]].columns))])
    screenerMetrics.append(test)

pd.DataFrame(screenerMetrics)


['JW-A']


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,PETS,57.0,19.09,3.963852,29.485672,36.75,19.09,3.078771,26.92028,21.85,...,1.550001,1.550001,11914879.587629,0.156285,0.072804,36.75,36.75,36.75,36.75,0.769559
1,AAWW,97.129997,38.360001,12.537162,68.06686,97.129997,62.130001,8.212463,77.57632,70.099998,...,3.369995,3.369995,10838428.041237,0.227714,0.540071,76.220001,95.0,97.129997,97.129997,0.941096
2,PAYX,141.919998,69.959999,19.64563,103.106836,141.919998,101.730003,8.677613,120.295488,125.599998,...,8.169998,8.169998,35785092.783505,0.593929,0.773207,105.169998,126.82,141.919998,141.919998,0.998737
3,TRUP,158.25,37.139999,22.44276,89.81224,158.25,54.68,20.491984,96.9575,67.580002,...,6.510002,6.510002,9204711.340206,0.124553,0.251342,126.525002,158.25,158.25,158.25,1.067627
4,AIG,65.730003,25.57,11.526413,46.637769,65.730003,44.540001,4.941777,56.03744,57.740002,...,0.93,0.93,106040269.896907,0.622935,0.801046,54.080002,62.540001,64.900002,65.730003,1.066169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,RGA,134.929993,66.989998,13.045893,109.251732,127.220001,94.32,6.151094,112.037061,125.300003,...,17.43,10.170006,10116717.525773,0.941641,0.858257,126.830002,126.830002,126.830002,126.830002,0.759841
1516,NVT,39.529999,16.92,6.771653,28.395447,39.529999,28.27,2.175311,34.01427,36.610001,...,2.98,2.98,20986736.082474,0.740675,0.870854,33.290001,38.369999,39.529999,39.529999,0.800482
1517,BC,117.620003,56.5,14.931305,85.480246,108.510002,66.400002,9.033974,92.192347,72.519997,...,2.899994,2.899994,15373767.42268,0.145334,0.262107,108.510002,108.510002,108.510002,108.510002,1.105019
1518,BDC,68.870003,28.549999,10.416104,48.355772,68.870003,45.310001,5.063406,56.782742,58.41,...,8.470001,4.0,5985481.649485,0.556027,0.740575,55.16,68.870003,68.870003,68.870003,0.901565


In [42]:
#calculateMetrics(npa[0])

In [43]:
#list(set(SP1500).difference([n for n in pd.DataFrame(screenerMetrics).set_index(0).index]))

In [44]:
#market_data.index = [sub.replace('.', '-') for sub in market_data.index]


In [45]:
set(np.array(market_data.index)).difference(pd.DataFrame(screenerMetrics).set_index(0).index)

set()

In [46]:
metrics = pd.concat([pd.DataFrame(screenerMetrics)],axis=0)

In [47]:
#np.min(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['Open'])

In [48]:
#calculateMinMax(npa[np.where(np.array(names_prices)=='GBX')[0][0]])

In [49]:
#pd.DataFrame(screener['ririsk_trend_factorsk_trend']).sort_values(by='risk_trend_factor',ascending=False)

In [50]:
prices_df = pd.concat([prices_df],axis=0)

In [51]:
npa = []
for f in FRED_Indicators:
    npa.append([f,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])

In [52]:
   
completed_fred = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(Fred_Data, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_fred = pd.concat([completed_fred,f.result()],axis=0)
client.close()

completed_fred_pvt = pd.pivot_table(completed_fred, values=['Value'], index=['DATE'],columns=['Symbol'])
completed_fred_pvt

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Value                                                   \
Symbol     ASPUS AWHAETP B432RG3A086NBEA BAA10Y BACDINA066MNFRBNY   
DATE                                                                
2020-06-09   NaN     NaN             NaN   2.80               NaN   
2020-06-10   NaN     NaN             NaN   2.86               NaN   
2020-06-11   NaN     NaN             NaN   2.91               NaN   
2020-06-12   NaN     NaN             NaN   2.91               NaN   
2020-06-13   NaN     NaN             NaN    NaN               NaN   
...          ...     ...             ...    ...               ...   
2022-06-05   NaN     NaN             NaN    NaN               NaN   
2022-06-06   NaN     NaN             NaN   2.08               NaN   
2022-06-07   NaN     NaN             NaN   2.07               NaN   
2022-06-08   NaN     NaN             NaN   2.07               NaN   
2022-06-09   NaN     NaN             NaN    NaN               NaN   

                                                                       \
Symbol     BACTSAMFRBDAL BAMLC0A0CM BAMLCC0A1AAATRIV BAMLCC0A4BBBTRIV   
DATE                                                                    
2020-06-09           NaN       1.59           771.40           945.70   
2020-06-10           NaN       1.61           775.57           948.41   
2020-06-11           NaN       1.70           780.02           947.55   
2020-06-12           NaN       1.69           775.56           944.65   
2020-06-13           NaN        NaN              NaN              NaN   
...                  ...        ...              ...              ...   
2022-06-05           NaN        NaN              NaN              NaN   
2022-06-06           NaN       1.37           665.39           875.44   
2022-06-07           NaN       1.38           669.75           878.93   
2022-06-08           NaN       1.37           666.36           876.30   
2022-06-09           NaN        NaN              NaN              NaN   

                        ...                                              \
Symbol     BAMLH0A3HYC  ... USROE USSTHPI VIXCLS VXVCLS WDFUELLA WGS3MO   
DATE                    ...                                               
2020-06-09       13.29  ...   NaN     NaN  27.57  29.67      NaN    NaN   
2020-06-10       13.43  ...   NaN     NaN  27.57  29.57      NaN    NaN   
2020-06-11       14.06  ...   NaN     NaN  40.79  40.43      NaN    NaN   
2020-06-12       13.96  ...   NaN     NaN  36.09  38.05    1.169   0.17   
2020-06-13         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   
...                ...  ...   ...     ...    ...    ...      ...    ...   
2022-06-05         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   
2022-06-06        9.68  ...   NaN     NaN  25.07  27.22      NaN    NaN   
2022-06-07        9.84  ...   NaN     NaN  24.02  26.52      NaN    NaN   
2022-06-08        9.85  ...   NaN     NaN  23.96  26.54      NaN    NaN   
2022-06-09         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   

                                                        
Symbol     WILLLRGCAPVAL WILLMICROCAPPR WPU0911 WTB3MS  
DATE                                                    
2020-06-09      90990.35       11984.24     NaN    NaN  
2020-06-10      89903.67       11711.97     NaN    NaN  
2020-06-11      84391.30       10754.28     NaN    NaN  
2020-06-12      85653.67       11094.02     NaN   0.17  
2020-06-13           NaN            NaN     NaN    NaN  
...                  ...            ...     ...    ...  
2022-06-05           NaN            NaN     NaN    NaN  
2022-06-06     122761.95       13859.87     NaN    NaN  
2022-06-07     124066.46       14167.07     NaN    NaN  
2022-06-08     122467.62       14120.63     NaN    NaN  
2022-06-09           NaN            NaN     NaN    NaN  

[731 rows x 128 columns]

In [53]:
npa = []
for e in etf_bonds:
    npa.append([e,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
completed_bonds = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(dl, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_bonds = pd.concat([completed_bonds,f.result()],axis=0)
client.close()

completed_bonds_pvt = pd.pivot_table(completed_bonds, values='Close', index=['Date'],columns=['Symbol'])
completed_bonds_pvt

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Symbol,AGG,AGZ,BIL,GBF,GOVT,IEF,IEI,ILTB,IMTB,ISTB,IUSB,LQD,MEAR,MUB,NEAR,SHV,SHY,SUB,TLT
Date,,,,,,,,,,,,,,,,,,,
2020-06-09,114.796921,118.086929,91.527000,122.024231,27.167822,119.265274,131.719513,70.750366,50.829491,50.326748,52.626720,128.930511,49.784447,112.444061,49.234745,110.498024,86.098709,106.734375,156.049530
2020-06-10,115.267159,118.548164,91.537003,122.605865,27.285515,120.157333,132.205215,71.512268,50.907532,50.414936,52.870682,129.777573,49.804348,112.493004,49.294090,110.498024,86.148468,106.813522,158.363342
2020-06-11,115.071236,118.450012,91.516998,122.576286,27.413017,120.583504,132.244858,71.194000,50.985577,50.297348,52.685276,128.229462,49.704857,112.659431,49.224846,110.517990,86.108650,106.714600,161.356522
2020-06-12,115.188797,118.734619,91.537003,122.487579,27.344360,120.315918,132.175461,71.319374,50.897778,50.297348,52.743824,128.852631,49.802361,112.493004,49.274315,110.517990,86.138512,106.585960,159.800888
2020-06-15,115.570854,118.587418,91.537003,123.010040,27.344360,120.216789,132.125900,71.917328,50.966076,50.414936,52.968273,130.683105,49.759575,112.688797,49.294090,110.498024,86.118614,106.645340,159.889496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-02,102.989998,110.529999,91.440002,108.849998,24.049999,102.760002,119.790001,57.410000,45.560001,47.840000,47.630001,113.550003,49.764999,107.949997,49.430000,110.169998,83.150002,104.800003,116.279999
2022-06-03,102.820000,110.379997,91.419998,108.639999,24.020000,102.559998,119.620003,57.189999,45.494999,47.810001,47.549999,113.190002,49.779999,107.989998,49.419998,110.190002,83.120003,104.839996,116.029999
2022-06-06,102.190002,109.980003,91.430000,108.059998,23.860001,101.790001,119.139999,56.549999,45.220001,47.700001,47.279999,112.389999,49.735001,107.720001,49.410000,110.169998,83.010002,104.790001,113.889999


In [54]:
#return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend_factor,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,,pct_2yr])
metrics.columns = ['Symbol','max_2y','min_2y','std_2y','mean_2y','max_1y','min_1y','std_1y','mean_1y','latest_p','2yr_close_0pct','2yr_close_2pct','2yr_close_9pct','2yr_close_25pct','2yr_close_50pct','2yr_close_75pct','2yr_close_91pct','2yr_close_98pct','2yr_close_100pct','1yr_close_0pct','1yr_close_2pct','1yr_close_9pct','1yr_close_25pct','1yr_close_50pct','2yr_close_75pct','1yr_close_91pct','1yr_close_98pct','1yr_close_100pct','risk_trend_factor','supply_trend_1yr','supply_trend_180d','supply_trend_90d','supply_trend_60d','supply_trend_30d','vol_30d_2yr','pct_1yr','pct_2yr','MIN_Lookback_One_Year', 'MIN_Lookback_6_Months', 'MIN_Lookback_2_Months', 'MIN_Lookback_1_Months','volume_factor']

In [55]:
completed_bonds

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2020-06-09,112.267847,112.561540,112.189528,112.444061,3125800,MUB
2020-06-10,112.512580,112.571315,112.355942,112.493004,1458600,MUB
2020-06-11,112.571321,112.825854,112.551745,112.659431,951100,MUB
2020-06-12,112.620267,112.649635,112.444053,112.493004,812800,MUB
2020-06-15,112.581110,112.737748,112.541951,112.688797,753000,MUB
...,...,...,...,...,...,...
2022-06-02,119.830002,119.860001,119.589996,119.790001,1483800,IEI
2022-06-03,119.510002,119.690002,119.510002,119.620003,790400,IEI
2022-06-06,119.459999,119.470001,119.099998,119.139999,2057300,IEI


In [56]:
screener = metrics
a = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend_factor'].apply(type) != np.str_].sort_values(by='risk_trend_factor',ascending=False)
b = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend_factor'].apply(type) == np.str_]
screener_sorted = pd.concat([a,b],axis=0)


In [57]:
screener_sorted

,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,2yr_close_0pct,...,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr,MIN_Lookback_One_Year,MIN_Lookback_6_Months,MIN_Lookback_2_Months,MIN_Lookback_1_Months,volume_factor
Symbol,,,,,,,,,,,,,,,,,,,,,
MTOR,36.299999,18.16,4.588549,26.92444,36.299999,20.5,5.415257,27.78056,36.119999,18.219999,...,0.529999,0.139999,15273304.742268,0.988608,0.990077,32.93,32.93,36.240002,36.279999,1.652223
Y,862.869995,460.579987,89.268635,649.33134,862.869995,585.099976,75.10839,704.020081,834.159973,467.079987,...,1.789978,1.789978,1613947.42268,0.896641,0.928634,737.890015,737.890015,862.869995,862.869995,1.634502
LNTH,73.779999,10.52,14.907606,26.16954,73.779999,22.200001,15.395024,35.9942,72.089996,10.75,...,13.329998,12.959995,13932232.371134,0.967235,0.973285,25.790001,31.6,60.709999,69.738998,1.716236
TVTY,32.889999,10.19,5.641415,22.84244,32.889999,21.25,3.262427,26.74488,32.169998,9.65,...,0.539999,0.539999,10500521.443299,0.938144,0.968282,27.85,29.18,32.889999,32.889999,1.584759
CVX,182.399994,65.160004,29.395132,104.858891,182.399994,92.860001,28.034779,124.876618,181.130005,61.47121,...,23.210007,16.940002,239020305.360825,0.985817,0.989168,113.110001,119.260002,174.759995,174.759995,0.924514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EMBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,24.780001,...,2.549999,2.549999,32352451.428571,NaN,NaN,NaN,NaN,NaN,NaN,0.530383
ESAB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.400002,44.880001,...,1.740002,1.740002,10846273.333333,NaN,NaN,NaN,NaN,NaN,NaN,0.870087
AIRC,NaN,NaN,NaN,NaN,55.82,42.305,2.838424,50.962,42.580002,36.15271,...,-0.939999,-0.939999,17079748.725212,0.020348,NaN,NaN,NaN,55.529999,55.82,1.238677


In [58]:
writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')

prices_df.to_excel(writer, sheet_name='SP1500_Sectors_Indexes_Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

#sectors_df.to_excel(writer, sheet_name='stock_sectors')

#mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_sorted.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"